# Python Code TAO Assignment 5

- `Romica Raisinghani | 2021101053`
- `Sriteja Reddy Pashya | 2021111019`

## Imports

In [10]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

## Code

1. Initialization:

`def __init__(self, grid_size, num_spiders, num_flies)`:
- Initialization of the simulation environment with grid size, number of spiders, and flies.
- Creation of a 2D grid representing the environment.
-  Random placement of spiders and flies on the grid.

2. Initialize Grid State from File

`    def read_initial_state_from_file(self, file_path)`:
- Reads the initial state of a 2D grid from a specified text file.
Each character in the file represents a grid cell, with 'S' for spiders and 'F' for flies.
- Resets the grid to zeros before reading the new state, ensuring a clean starting point.
- Offers flexibility in setting up specific scenarios for the simulation by allowing the initial state to be defined externally in a file.

3. Placing Spiders and Flies

`def place_spiders_and_flies(self)`:
- Randomly places spiders (1) and flies (2) on the grid.
- Ensures unique positions for spiders and flies.

4. Base Policy Movement

`def base_policy_move(self)`:
- Implements the base policy for spider movement.
- Each spider moves one square towards its nearest fly with a horizontal preference.

5. Multiagent Rollout Movement

`def multiagent_rollout_move(self)`:
- Implements the multiagent rollout strategy for spider movement.
- Spiders move one at a time towards their nearest untargeted fly, avoiding double targeting.
- Updates the grid to reflect the new positions of spiders.

6. Finding Nearest Untargeted Fly

`def find_nearest_untargeted_fly(self, spider_pos, targeted_flies)`:
- Finds the nearest untargeted fly to a given spider position.
- Considers only flies that have not been targeted by previous moves.
- Returns the position of the nearest untargeted fly.


7. Updating Grid for Animation

`def update_grid(self, frame, strategy, im)`:
- Updates the grid based on the chosen strategy (base policy or multiagent rollout) for animation.
- Updates the image with the new grid state.
- Returns the updated image array.

8. Animation

`def animate(self, strategy)`:
- Animates the spider and fly movement based on the chosen strategy (base policy or multiagent rollout).
- Creates an animation showing the grid state at each move.
- Returns the total number of moves taken.

9. Running Simulation

`def run_simulation(self, strategy)`:
- Runs the simulation until all flies are caught using the specified strategy (base policy or multiagent rollout).
- Prints the initial and final states of the grid, along with the total number of moves.

10. Manhattan Distance Calculation

`def manhattan_distance(self, point1, point2)`:
- Calculates the Manhattan distance between two points.



In [11]:
class SpidersAndFlies:
    def __init__(self, grid_size, num_spiders, num_flies):
        self.grid_size = grid_size
        self.num_spiders = num_spiders
        self.num_flies = num_flies
        self.grid = np.zeros((grid_size, grid_size))
        self.place_spiders_and_flies()

    def read_initial_state_from_file(self, file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Reset the grid before reading the new state
            self.grid = np.zeros((self.grid_size, self.grid_size))
            for y, line in enumerate(lines):
                for x, char in enumerate(line.strip()):
                    if char == 'S':
                        self.grid[y, x] = 1  # Spider
                    elif char == 'F':
                        self.grid[y, x] = 2  # Fly

    def place_spiders_and_flies(self):
        # Randomly place spiders (represented by 1) and flies (represented by 2)
        positions = set()
        while len(positions) < self.num_spiders + self.num_flies:
            positions.add((random.randint(0, self.grid_size-1), random.randint(0, self.grid_size-1)))

        positions = list(positions)
        for i in range(self.num_spiders):
            self.grid[positions[i]] = 1

        for i in range(self.num_spiders, self.num_spiders + self.num_flies):
            self.grid[positions[i]] = 2

    def find_nearest_fly(self, spider_pos):
        min_distance = float('inf')
        nearest_fly_pos = None
        for y in range(self.grid_size):
            for x in range(self.grid_size):
                if self.grid[y, x] == 2:  # Fly found
                    distance = self.manhattan_distance(spider_pos, (y, x))
                    if distance < min_distance:
                        min_distance = distance
                        nearest_fly_pos = (y, x)
        return nearest_fly_pos

    def base_policy_move(self):
        for spider_y in range(self.grid_size):
            for spider_x in range(self.grid_size):
                if self.grid[spider_y, spider_x] == 1:  # Spider found
                    nearest_fly = self.find_nearest_fly((spider_y, spider_x))
                    if nearest_fly:
                        dy, dx = nearest_fly[0] - spider_y, nearest_fly[1] - spider_x
                        move_y, move_x = 0, 0
                        if abs(dx) > abs(dy) or (abs(dx) == abs(dy) and dx != 0):  # Horizontal preference
                            move_x = 1 if dx > 0 else -1
                        else:
                            move_y = 1 if dy > 0 else -1
                        # Update spider's position
                        self.grid[spider_y, spider_x] = 0  # Remove spider from old position
                        self.grid[spider_y + move_y, spider_x + move_x] = 1  # Place spider at new position


    # def standard_rollout_move(self):
    #     moves = []
    #     for spider_y in range(self.grid_size):
    #         for spider_x in range(self.grid_size):
    #             if self.grid[spider_y, spider_x] == 1:
    #                 nearest_fly = self.find_nearest_fly((spider_y, spider_x))
    #                 if nearest_fly:
    #                     # Calculate move as in base_policy_move
    #                     # Store moves in the moves list
    #                     pass
    #     # Update grid with all moves
    #     for spider_y, spider_x, move_y, move_x in moves:
    #         self.grid[spider_y, spider_x] = 0
    #         self.grid[spider_y + move_y, spider_x + move_x] = 1

    def multiagent_rollout_move(self):
        # Get the positions of all spiders
        spider_positions = [(y, x) for y in range(self.grid_size) for x in range(self.grid_size) if self.grid[y, x] == 1]
        
        # Track the flies that are targeted by a move to avoid double targeting
        targeted_flies = set()

        for spider_pos in spider_positions:
            # Find the nearest fly that is not already being targeted
            nearest_fly = self.find_nearest_untargeted_fly(spider_pos, targeted_flies)
            if nearest_fly:
                dy, dx = nearest_fly[0] - spider_pos[0], nearest_fly[1] - spider_pos[1]
                move_y, move_x = 0, 0
                if abs(dx) > abs(dy) or (abs(dx) == abs(dy) and dx != 0):
                    move_x = 1 if dx > 0 else -1
                else:
                    move_y = 1 if dy > 0 else -1
                self.grid[spider_pos[0], spider_pos[1]] = 0
                new_pos = (spider_pos[0] + move_y, spider_pos[1] + move_x)
                # Check if not moving into another spider
                if self.grid[new_pos] != 1:
                    self.grid[new_pos] = 1
                    # Mark the fly as targeted for this turn
                    targeted_flies.add(nearest_fly)
                else:
                    # If the position is already occupied, place the spider back to its original position
                    self.grid[spider_pos[0], spider_pos[1]] = 1

    def find_nearest_untargeted_fly(self, spider_pos, targeted_flies):
        min_distance = float('inf')
        nearest_fly_pos = None
        for y in range(self.grid_size):
            for x in range(self.grid_size):
                if self.grid[y, x] == 2 and (y, x) not in targeted_flies:  # Fly found and not targeted
                    distance = self.manhattan_distance(spider_pos, (y, x))
                    if distance < min_distance:
                        min_distance = distance
                        nearest_fly_pos = (y, x)
        return nearest_fly_pos
    
    def update_grid(self, frame, strategy, im):
        if strategy == 'base_policy':
            self.base_policy_move()
        # elif strategy == 'standard_rollout':
        #     self.standard_rollout_move()
        elif strategy == 'multiagent_rollout':
            self.multiagent_rollout_move()
        
        # Update the image with the new grid state
        im.set_array(self.grid)
        return [im]

    def animate(self, strategy):
        fig, ax = plt.subplots()
        im = ax.imshow(self.grid, cmap='hot', interpolation='nearest', vmin=0, vmax=2)
        
        # Initialize the move counter outside the update function
        move_counter = [0]

        def update(frame):
            # Only make a move if there are still flies
            if np.any(self.grid == 2):
                self.update_grid(frame, strategy, im)
                move_counter[0] += 1  # Increment the move counter
                ax.set_title(f"Move: {move_counter[0]}")
            else:
                # Stop the animation if there are no flies left
                ani.event_source.stop()
                print(f"Total moves taken: {move_counter[0]}")
                plt.close(fig)  # Close the figure window

            return [im]

        # Create the animation
        ani = animation.FuncAnimation(fig, update, frames=range(100), interval=1000, blit=False, repeat=False)

        plt.show()
        return move_counter[0]  # Return the number of moves taken



    def run_simulation(self, strategy):
        # Save the initial state
        initial_state = np.copy(self.grid)
        print("Initial Board:")
        print(initial_state)

        moves = 0
        while np.any(self.grid == 2):  # Continue until all flies are caught
            if strategy == 'base_policy':
                self.base_policy_move()
            # elif strategy == 'standard_rollout':
            #     self.standard_rollout_move()
            elif strategy == 'multiagent_rollout':
                self.multiagent_rollout_move()
            moves += 1

        print(f"Total moves with {strategy}: {moves}")

        print("Final Board:")
        print(self.grid)
    
    def manhattan_distance(self, point1, point2):
        return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])

In [12]:

print(matplotlib.get_backend())
matplotlib.use('TkAgg')

simulation = SpidersAndFlies(grid_size=10, num_spiders=2, num_flies=5)
# simulation.read_initial_state_from_file('input_1.txt')
# After reading the initial state, you can run the simulation or animation
# simulation.run_simulation(strategy='multiagent_rollout')
simulation.animate(strategy='multiagent_rollout')


TkAgg


14